<a href="https://colab.research.google.com/github/DivineOmega/clip-interrogator/blob/main/clip_interrogator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLIP Interrogator by [@pharmapsychotic](https://twitter.com/pharmapsychotic) 

<br>

What do the different OpenAI CLIP models see in an image? What might be a good text prompt to create similar images using CLIP guided diffusion or another text to image model? The CLIP Interrogator is here to get you answers!

<br>

If this notebook is helpful to you please consider buying me a coffee via [ko-fi](https://ko-fi.com/pharmapsychotic) or following me on [twitter](https://twitter.com/pharmapsychotic) for more cool Ai stuff. 🙂

And if you're looking for more Ai art tools check out my [Ai generative art tools list](https://pharmapsychotic.com/tools.html).


In [1]:
#@title Check GPU
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-27fdc045-488a-714f-6421-8ab5ecb2e293)


In [32]:
#@title Setup
!pip3 install ftfy regex tqdm transformers==4.15.0 timm==0.4.12 fairscale==0.4.4
!pip3 install git+https://github.com/openai/CLIP.git
!git clone https://github.com/pharmapsychotic/clip-interrogator.git

import clip
import gc
import numpy as np
import os
import pandas as pd
import requests
import torch
import torchvision.transforms as T
import torchvision.transforms.functional as TF

from IPython.display import display
from PIL import Image
from torch import nn
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def load_list(filename):
    with open(filename, 'r', encoding='utf-8', errors='replace') as f:
        items = [line.strip() for line in f.readlines()]
    return items

def rank(model, image_features, text_array, top_count=1):
    top_count = min(top_count, len(text_array))
    text_tokens = clip.tokenize([text for text in text_array]).cuda()
    with torch.no_grad():
        text_features = model.encode_text(text_tokens).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)

    similarity = torch.zeros((1, len(text_array))).to(device)
    for i in range(image_features.shape[0]):
        similarity += (100.0 * image_features[i].unsqueeze(0) @ text_features.T).softmax(dim=-1)
    similarity /= image_features.shape[0]

    top_probs, top_labels = similarity.cpu().topk(top_count, dim=-1)  
    return [(text_array[top_labels[0][i].numpy()], (top_probs[0][i].numpy()*100)) for i in range(top_count)]

def interrogate(image, models):

    if len(models) != 1:
        print("You must select exactly one model.")
        return

    table = []
    #bests = [[('',0)]]*5

    model_name = models[0]

    print(f"Interrogating with {model_name}...")
    model, preprocess = clip.load(model_name)
    model.cuda().eval()

    images = preprocess(image).unsqueeze(0).cuda()
    with torch.no_grad():
        image_features = model.encode_image(images).float()
    image_features /= image_features.norm(dim=-1, keepdim=True)

    ranks = [
        rank(model, image_features, [f"{x} photo of a person" for x in focal_lengths]),
        rank(model, image_features, [f"photo of a {x} person" for x in styles]),
        rank(model, image_features, [f"photo of a {x} person" for x in ages]),
        rank(model, image_features, [f"photo of a {x}" for x in genders]),
        rank(model, image_features, [f"photo of a person with {x} length hair" for x in hair_lengths]),
        rank(model, image_features, [f"photo of a person with {x} coloured hair" for x in hair_colours]),
        rank(model, image_features, [f"photo of a person with a {x} shaped_face" for x in face_shapes]),
        rank(model, image_features, [f"photo of a person with {x} eyes" for x in eye_colours]),
        rank(model, image_features, [f"photo of a person wearing a {x}" for x in top_garments]),
        rank(model, image_features, [f"photo of a person wearing a {x}" for x in bottom_garments]),
        rank(model, image_features, [f"photo of a person wearing a {x}" for x in outer_garments]),
        rank(model, image_features, [f"photo of a person with a {x} background" for x in backgrounds]),
        rank(model, image_features, [f"photo of a person wearing {x}" for x in glasses]),
        rank(model, image_features, [f"photo of a person {x}" for x in poses]),
        rank(model, image_features, [f"{x} photo of a person" for x in photo_styles]),
        rank(model, image_features, [f"photo of a {x} person" for x in nationalities])
        # rank(model, image_features, flavors, top_count=3)
    ]

    #print (ranks[5])

    #for i in range(len(ranks)):
    #    confidence_sum = 0
    #    for ci in range(len(ranks[i])):
    #        confidence_sum += ranks[i][ci][1]
    #    if confidence_sum > sum(bests[i][t][1] for t in range(len(bests[i]))):
    #        bests[i] = ranks[i]

    #row = [model_name]
    #for r in ranks:
    #    row.append(', '.join([f"{x[0]} ({x[1]:0.1f}%)" for x in r]))

    #table.append(row)

    gc.collect()

    #display(pd.DataFrame(table, columns=["Model", "Focal Length", "Style", "Age", "Gender", "Hair Length", "Hair Colour", "Face Shape", "Eye Colour", "Top Garment", "Bottom Garment", "Outer Garment"]))

    focal_length = find_original_attribute(ranks[0][0][0], focal_lengths)
    style = find_original_attribute(ranks[1][0][0], styles)
    age = find_original_attribute(ranks[2][0][0], ages)
    gender = find_original_attribute(ranks[3][0][0], genders)
    hair_length = find_original_attribute(ranks[4][0][0], hair_lengths)
    hair_colour = find_original_attribute(ranks[5][0][0], hair_colours)
    face_shape = find_original_attribute(ranks[6][0][0], face_shapes)
    eye_colour = find_original_attribute(ranks[7][0][0], eye_colours)
    top_garment = find_original_attribute(ranks[8][0][0], top_garments)
    bottom_garment = find_original_attribute(ranks[9][0][0], bottom_garments)
    outer_garment = find_original_attribute(ranks[10][0][0], outer_garments)
    background = find_original_attribute(ranks[11][0][0], backgrounds)
    glasses_singular = find_original_attribute(ranks[12][0][0], glasses)
    pose = find_original_attribute(ranks[13][0][0], poses)
    photo_style = find_original_attribute(ranks[14][0][0], photo_styles)
    nationality = find_original_attribute(ranks[15][0][0], nationalities)

    garments = []
    if (top_garment != 'no top garment'):
        garment_colour_rank = rank(model, image_features, [f"photo of a person wearing a {x} {top_garment}" for x in garment_colours])
        garment_colour = find_original_attribute(garment_colour_rank[0][0], garment_colours)
        garments.append('a ' + garment_colour + ' ' + top_garment)
    if (bottom_garment != 'no bottom garment'):
        garment_colour_rank = rank(model, image_features, [f"photo of a person wearing a {x} {bottom_garment}" for x in garment_colours])
        garment_colour = find_original_attribute(garment_colour_rank[0][0], garment_colours)
        garments.append('a ' + garment_colour + ' ' +bottom_garment)
    if (outer_garment != 'no outer garment'):
        garment_colour_rank = rank(model, image_features, [f"photo of a person wearing a {x} {outer_garment}" for x in garment_colours])
        garment_colour = find_original_attribute(garment_colour_rank[0][0], garment_colours)
        garments.append('a ' + garment_colour + ' ' + outer_garment)
    if (glasses_singular != 'no glasses'):
        garment_colour_rank = rank(model, image_features, [f"photo of a person wearing {x} {glasses_singular}" for x in garment_colours])
        garment_colour = find_original_attribute(garment_colour_rank[0][0], garment_colours)
        garments.append(garment_colour + ' ' + glasses_singular)

    wearing = ''
    if (len(garments) > 0):
        wearing += 'wearing ';
        for i, garment in enumerate(garments, start=0):
            wearing += garment
            if (i < len(garments)-1):
              wearing += ' and '

    print(f"\n\n{focal_length} {photo_style} photo of a {style} {age} {nationality} {gender} {pose} with {hair_length} {hair_colour} hair, a {face_shape} face and {eye_colour} eyes {wearing}. {background} background. dof. bokeh. extremely detailed. Nikon D850.")

def find_original_attribute(best, original_attributes):
  for original_attribute in original_attributes:
        if (' '+original_attribute+' ' in ' '+best+' '):
            return original_attribute
            

data_path = "../clip-interrogator/data/"

focal_lengths = ['18mm', '24mm', '35mm', '55mm', '85mm', '105mm', '135mm', '200mm'];
styles = ['tattoed', 'professional', 'beautiful', 'attractive','sexy naked', 'sexy', 'naked', 'ugly', 'average looking', 'stylish', 'glamourous', 'elegant', 'cyberpunk', 'steampunk', 'handsome', 'goth', 'emo', 'alternative', 'punk', 'pretty', 'cute']
ages = ['18 year old', '20 year old', '30 year old', '40 year old', '50 year old', '60 year old', '70 year old', '80 year old', '90 year old', '100 year old']
genders = ['woman', 'man']
hair_lengths = ['short bob', 'medium bob', 'long bob', 'long pony tail', 'pony tail','long straight', 'straight shoulder length', 'straight waist length', 'long wavey', 'wavey shoulder length', 'wavey waist length', 'waist length', 'shoulder length', 'pixie cut', 'very long', 'very short', 'long', 'medium', 'short']
hair_colours = ['dirty blond', 'bleach blond', 'blond', 'light brown', 'dark brown', 'brown', 'black', 'light red', 'bright red', 'dark red', 'red', 'ginger', 'very white', 'white', 'grey', 'gray', 'green', 'blue', 'cyan', 'orange', 'purple', 'pink', 'gold']
face_shapes = ['diamond', 'heart', 'oval', 'rectangular', 'round', 'square']
eye_colours = ['dark brown', 'chocolate brown', 'light brown', 'light blue', 'dark blue', 'light green', 'dark green', 'blue', 'brown', 'grey', 'gray', 'green']
top_garments = ['no top garment', 'Anna (Frozen) dress', 'Elsa (Frozen) dress', 'pink and blue patterned top' 'sleveless button down top', 'sleveless top', 'summer dress', 'military style top', 'headscarf', 'hijab', 'hooded garment', 'long dress', 'short dress', 'tank top', 'tube top', 'crop top', 'halter top', 'off-the-shoulder top', 'one-shoulder top', 't-shirt', 'blouse', 'button-down shirt', 'polo shirt', 'bikini', 'bra', 'shirt and tie', 'shirt', 'catsuit', 'dress', 'rags', 'princess dress', 'wedding dress']
bottom_garments = ['no bottom garment', 'fishnet tights', 'denim jeans', 'short denim shorts', 'short ripped denim shorts', 'short denim dress', 'denim shorts', 'denim dress', 'skirt', 'short skirt', 'long skirt', 'skort', 'shorts', 'short shorts', 'long shorts', 'jeans', 'capris', 'pants', 'trousers', 'tutu']
outer_garments = ['no outer garment', 'fancy dress', 'super hero costume', 'wonder woman costume', 'superman constume', 'american flag', 'united kingdom flag', 'flag', 'coat', 'cardigan', 'jacket', 'hoodie', 'robe', 'suit jacket', 'sweater']
glasses = ['no glasses', 'cat eye contact lenses', 'contact lenses', 'round glasses', 'square glasses', 'rectangular glasses', 'cat eye glasses', 'sunglasses', 'glasses']
garment_colours = ['red with small white flowers', 'invisible', 'partially visible', 'transparent', 'semi-transparent', 'see through', 'black latex', 'latex', 'yellow and black tiger print', 'light pink', 'light blue', 'dark blue', 'black and white horizontally striped', 'silver and gold metallic', 'polka dot', 'dotted', 'horizontally striped', 'vertically striped', 'stripped', 'cosplay', 'black and white', 'leopard print', 'tiger print', 'lion print', 'animal print', 'traditional', 'silver metallic', 'red metallic', 'green metallic', 'blue metallic', 'metallic', 'silver sequinned', 'sequinned', 'intricately patterned', 'frilly white', 'frilly black', 'frilly', 'geometric', 'patterned', 'flowery', 'floral', 'blue leather', 'black leather', 'leather', 'red', 'green', 'blue', 'white', 'black', 'orange', 'yellow', 'purple', 'pink', 'grey', 'gray', 'brown', 'gold', 'cyan']
backgrounds = ['snowy', 'mountain top', 'clothes shop', 'leaves on the ground', 'night time street light', 'roof', 'train station', 'British street', 'American street', 'street', 'inside of a car', 'crowd of people', 'sunset', 'building alongside a river', 'building', 'river', 'water', 'desert', 'grassy field', 'field of flowers', 'field', 'roof top', 'cyberpunk', 'steampunk', 'science fiction', 'city', 'industrial', 'white wall', 'brick wall', 'white kitchen', 'park', 'office', 'bedroom', 'kitchen', 'white', 'black', 'gray', 'grey', 'red', 'yellow', 'light blue', 'blue', 'cyan', 'orange', 'colourful', 'light', 'dark']
poses = ['holding a flower', 'posing with hands on hips', 'posing with one hand pointing at the camera', 'posing with hands on lap', 'hands on lap', 'playing a violin', 'playing a musical instrument', 'smoking a cigar', 'sitting at a desk writing on a sheet of paper', 'writing on a sheet of paper', 'sitting at a desk', 'eyes closed and arms down and out to the side', 'eyes closed and arms out to the side', 'yoga pose', 'arms crossed', 'smiling and looking directly at the camera', 'face palming', 'posing with a happy smile', 'standing with a happy smile', 'sitting with arms on legs with a happy smile', 'sitting with a happy smile', 'posing with both hands in hair', 'posing with one hand in hair', 'pulling a funny face and holding fingers up like horns above head', 'holding fingers up like horns above head', 'pulling a funny face', 'looking at the camera with mouth open', 'posing with eyes closed', 'standing posing with their mobile phone', 'posing with their mobile phone', 'smoking', 'posing with tongue sticking out', 'posing with chest forward', 'posing seductively', 'posing with finger on lips', 'posing with finger on face', 'resting', 'relaxing against a wall', 'standing against a wall', 'relaxing', 'sitting cross legged', 'standing', 'sitting', 'lying', 'stretching', 'exercising', 'kissing', 'walking', 'running', 'riding a bike', 'riding a house', 'driving a car', 'punching', 'kicking', 'fighting', 'posing', 'pouting']
photo_styles = ['selfie', 'side portrait', 'portrait', 'extreme close up', 'close up', 'full body', 'wide angle', 'distant', 'zoomed in', 'zoomed out']
nationalities = ['Afghan', 'Albanian', 'Algerian', 'American', 'Andorran', 'Angolan', 'Antiguans', 'Argentinean', 'Armenian', 'Australian', 'Austrian', 'Azerbaijani', 'Bahamian', 'Bahraini', 'Bangladeshi', 'Barbadian', 'Barbudans', 'Batswana', 'Belarusian', 'Belgian', 'Belizean', 'Beninese', 'Bhutanese', 'Bolivian', 'Bosnian', 'Brazilian', 'British', 'Bruneian', 'Bulgarian', 'Burkinabe', 'Burmese', 'Burundian', 'Cambodian', 'Cameroonian', 'Canadian', 'Cape Verdean', 'Central African', 'Chadian', 'Chilean', 'Chinese', 'Colombian', 'Comoran',  'Congolese', 'Costa Rican', 'Croatian', 'Cuban', 'Cypriot', 'Czech', 'Danish', 'Djibouti', 'Dominican', 'Dutch', 'Dutchman', 'Dutchwoman', 'East Timorese', 'Ecuadorean', 'Egyptian', 'Emirian', 'Equatorial Guinean', 'Eritrean', 'Estonian', 'Ethiopian', 'Fijian', 'Filipino', 'Finnish', 'French', 'Gabonese', 'Gambian', 'Georgian', 'German', 'Ghanaian', 'Greek', 'Grenadian', 'Guatemalan', 'Guinea-Bissauan', 'Guinean', 'Guyanese', 'Haitian', 'Herzegovinian', 'Honduran', 'Hungarian', 'I-Kiribati', 'Icelander', 'Indian', 'Indonesian', 'Iranian', 'Iraqi', 'Irish', 'Israeli', 'Italian', 'Ivorian', 'Jamaican', 'Japanese', 'Jordanian', 'Kazakhstani', 'Kenyan', 'Kittian and Nevisian', 'Kuwaiti', 'Kyrgyz', 'Laotian', 'Latvian', 'Lebanese', 'Liberian', 'Libyan', 'Liechtensteiner', 'Lithuanian', 'Luxembourger', 'Macedonian', 'Malagasy', 'Malawian', 'Malaysian', 'Maldivan', 'Malian', 'Maltese', 'Marshallese', 'Mauritanian', 'Mauritian', 'Mexican', 'Micronesian', 'Moldovan', 'Monacan', 'Mongolian', 'Moroccan', 'Mosotho', 'Motswana', 'Mozambican', 'Namibian', 'Nauruan', 'Nepalese', 'Netherlander', 'New Zealander', 'Ni-Vanuatu', 'Nicaraguan', 'Nigerian', 'Nigerien', 'North Korean', 'Northern Irish', 'Norwegian', 'Omani', 'Pakistani', 'Palauan', 'Panamanian', 'Papua New Guinean', 'Paraguayan', 'Peruvian', 'Polish', 'Portuguese', 'Qatari', 'Romanian', 'Russian', 'Rwandan', 'Saint Lucian', 'Salvadoran', 'Samoan', 'San Marinese', 'Sao Tomean', 'Saudi', 'Scottish', 'Senegalese', 'Serbian', 'Seychellois', 'Sierra Leonean', 'Singaporean', 'Slovakian', 'Slovenian', 'Solomon Islander', 'Somali', 'South African', 'South Korean', 'Spanish', 'Sri Lankan', 'Sudanese', 'Surinamer', 'Swazi', 'Swedish', 'Swiss', 'Syrian', 'Taiwanese', 'Tajik', 'Tanzanian', 'Thai', 'Togolese', 'Tongan', 'Trinidadian or Tobagonian', 'Tunisian', 'Turkish', 'Tuvaluan', 'Ugandan', 'Ukrainian', 'Uruguayan', 'Uzbekistani', 'Venezuelan', 'Vietnamese', 'Welsh', 'Yemenite', 'Zambian', 'Zimbabwean']

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-b43ckv_o
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-b43ckv_o
fatal: destination path 'clip-interrogator' already exists and is not an empty directory.


In [ ]:
#@title Interrogate!

#@markdown 

#@markdown #####**Image:**

image_path_or_url = "" #@param {type:"string"}

#@markdown 

#@markdown #####**CLIP models:**

#@markdown For [StableDiffusion](https://stability.ai/blog/stable-diffusion-announcement) you can just use ViTL14<br>
#@markdown For [DiscoDiffusion](https://colab.research.google.com/github/alembics/disco-diffusion/blob/main/Disco_Diffusion.ipynb) and 
#@markdown [JAX](https://colab.research.google.com/github/huemin-art/jax-guided-diffusion/blob/v2.7/Huemin_Jax_Diffusion_2_7.ipynb) enable all the same models here as you intend to use when generating your images

ViTB32 = False #@param{type:"boolean"}
ViTB16 = False #@param{type:"boolean"}
ViTL14 = True #@param{type:"boolean"}
ViTL14_336px = False #@param{type:"boolean"}
RN101 = False #@param{type:"boolean"}
RN50 = False #@param{type:"boolean"}
RN50x4 = False #@param{type:"boolean"}
RN50x16 = False #@param{type:"boolean"}
RN50x64 = False #@param{type:"boolean"}

models = []
if ViTB32: models.append('ViT-B/32')
if ViTB16: models.append('ViT-B/16')
if ViTL14: models.append('ViT-L/14')
if ViTL14_336px: models.append('ViT-L/14@336px')
if RN101: models.append('RN101')
if RN50: models.append('RN50')
if RN50x4: models.append('RN50x4')
if RN50x16: models.append('RN50x16')
if RN50x64: models.append('RN50x64')

if str(image_path_or_url).startswith('http://') or str(image_path_or_url).startswith('https://'):
    image = Image.open(requests.get(image_path_or_url, stream=True).raw).convert('RGB')
else:
    image = Image.open(image_path_or_url).convert('RGB')

thumb = image.copy()
thumb.thumbnail([256, 256])
display(thumb)

interrogate(image, models=models)
